## EasyAgent

This notebook is intended to showcase a very simple chat agent and allow you to build your own by modifying it

In [ ]:
# Install dependencies

!pip install openai

In [ ]:
# Import some libraries for sending emails. Replace these as necessary when you change the agent's functionality

import smtplib
from email.mime.text import MIMEText


# Import the necessary libraries for the agent

from openai import OpenAI
import os
from typing import Annotated
import json


# Import our own custom utilities

from utils import *

In [ ]:
# Load the configuration file. You can make your own or just replace the values below

from config import *


# Settings for the email component. Replace these as necessary when you change the agent's functionality

email = config.user_email
email_password = config.user_email_password
convo_length_limit = 20


# Settings for the agent. Modify these as desired. Remember to set your own API key

agent_model_name = "gpt-4.1"
agent_model_temperature = 0.2
openai_api_key = config.openai_api_key

In [ ]:
# Modify this cell as desired


# These are your custom functions that the agent will be able to execute
# You can replace them with whatever tools you want
# Just make sure all inputs and outputs are annotated as shown below 
# and that they've been added to the "agent_functions" list below

def list_transcripts() -> list[str]:
    """Lists all available transcripts.

    Returns:
        list[str]: A list of available transcripts.
    """
    transcripts = []
    for filename in os.listdir("transcripts"):
        name = filename[:-4]
        first_name, last_name, date = name.split("_")
        transcripts.append(f"{first_name} {last_name} ({date})")
    return transcripts


def load_transcript(
    first_name: Annotated[str, "The first name of the client."],
    last_name: Annotated[str, "The last name of the client."],
    date: Annotated[str, "The date of the call in dd-mm-yy format."],
    ) -> str:
    """Loads the transcript of call with a client.

    Args:
        first_name (str): The first name of the client.
        last_name (str): The last name of the client.
        date (str): The date of the call in dd-mm-yy format.

    Returns:
        str: The content of the transcript.
    """
    with open(f"transcripts/{first_name.lower()}_{last_name.lower()}_{date}.txt", "r") as file:
        transcript = file.read()
    
    return transcript


def get_policies() -> str:
    """Returns the policies of the company.

    Returns:
        str: The policies of the company.
    """
    with open("policies.txt", "r") as file:
        policies = file.read()
    
    return policies


def calculator(
        square_footage: Annotated[int, "The square footage of the area to be cleaned."],
        price_per_sqft: Annotated[float, "The price per square foot for cleaning."],
) -> float:
    return square_footage * price_per_sqft


def send_email(
        address: Annotated[str, "The recipient's email address."],
        subject: Annotated[str, "The email subject."],
        text: Annotated[str, "The email content."],
        ):
    """
    Sends an email with the given text to the specified address.

    Args:
        address (str): The recipient's email address.
        subject (str): The email subject.
        text (str): The email content.
    
    Returns:
        None
    """
    sender_email = email
    sender_password = email_password

    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    msg = MIMEText(text)
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = address

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, address, msg.as_string())
        print("Email sent successfully!")
        print("Subject:", subject)
        print("Text:", text)
    except Exception as e:
        print(f"Failed to send email: {e}")


# Any other functions you want to give the agent go here
# Remember to add them with their description to agent_functions below


# tool_schema is a JSON specification of a function--its description, and inputs and outputs with their types
# This is essential to tell the agent what tools it can use and how to use them
# tool_name_to_func maps the function the agent decides to call back to the actual function to use

agent_functions = [
    (list_transcripts, "Lists all available transcripts."),
    (load_transcript, "Loads the transcript of a call with a client."),
    (get_policies, "Returns the policies of the company."),
    (calculator, "Calculates the cost of a given square footage."),
    (send_email, "Sends an email with the given text to the specified address.")
]
tool_schema, tool_name_to_func = get_function_schema(agent_functions)


# Prompt that tells the agent what "role" they should play and what their task is

expert_agent_sys_message = (
    "You are an administrative assistant for Windy City Cleaning Company. "
    "You have access to a database of transcripts from calls with clients. "
    "You can load transcripts and company policies, calculate costs based on square footage, and send emails. "
    "You will be given a task and you should use the tools available to you to complete it. "
    "If asked to prepare a quote, start by referencing the policies of the company and any client transcripts. "
    "Make sure to ground your answer in the information you have. "
    "Wait for approval before sending an email!"
    )

In [ ]:
client = OpenAI(api_key=openai_api_key)

def run_agent(user_query):
    chat_history = [
        {"role": "system", "content": expert_agent_sys_message},
        {"role": "user", "content": user_query}
        ]

    for _ in range(convo_length_limit):

        # If the last item in the conversation was not the agent, call the agent

        if chat_history[-1]['role'] in ("user", "system", "tool"):
            response = client.chat.completions.create(
                model=agent_model_name,
                messages=chat_history,
                tools=tool_schema,
                temperature=agent_model_temperature
            )

            chat_history.append(dict(response.choices[0].message))
            
            if chat_history[-1]["content"] is not None:
                print("AGENT:\n" + chat_history[-1]["content"] + "\n\n")


        # If the last conversation item was the agent calling functions, 
        # get the function results using the arguments provided by the agent

        elif chat_history[-1]['tool_calls'] is not None:                
            for call in chat_history[-1]['tool_calls']:
                try:
                    function = tool_name_to_func.get(call.function.name, None)
                    kwargs = json.loads(call.function.arguments)

                    print("AGENT TOOL CALL:\n" + call.function.name + "\n" + str(kwargs) + "\n\n")

                    if function is not None:
                        response = function(**kwargs)
                    else:
                        response = f"There is no tool called '{call.function.name}'. The available tools are {list(self.tool_name_to_func.keys())}"
                except Exception as e:
                    response = f"Error calling function '{call.function.name}': {str(e)}"
                
                chat_history.append({
                    "role": "tool", 
                    "content": json.dumps(response),
                    "tool_call_id": call.id
                    })
                
                print("TOOL RESPONSE:\n" + str(response) + "\n\n")

        
        # Otherwise, the agent is passing the conversation back to the user who can either continue or terminate it

        else:
            chat_history.append({"role": "user", "content": input("Provide further input or enter nothing to end session: ")})

            print("USER:\n" + chat_history[-1]["content"] + "\n\n")

            if chat_history[-1]['content'] == "":
                break

    return chat_history

In [ ]:
chat_history = run_agent("Prepare an email to John Smith with a quote")